# Lab No 2: Data Manipulation and Working with Web Services

## 2 Challenges

# Challenge No 1

In [ ]:
# Here is the code for challenge 1, Lab 2

In [ ]:
#1.Using a Dictionary, create a dataframe (table), with at least 4 columns and more than 100 rows. How come you can create this among data from scratch without defining every single row of data?
import numpy as np  
import pandas as pd

np.random.seed(42)

num_rows = 150

data = {
    'Bromley': np.random.randint(5, 30, num_rows),
    'Croydon': np.random.randint(5, 30, num_rows),
    'Dagenham': np.random.randint(5, 30, num_rows),
    'Lewisham': np.random.randint(5, 30, num_rows)
}

df = pd.DataFrame(data)
print(df.head())

In [ ]:
subset_df = df.iloc[:30, :3]
print(subset_df)

In [ ]:
filtered_df = subset_df.loc[(subset_df > 10).all(axis=1)]
filtered_C_df = filtered_df.loc[:,'Croydon']

print(filtered_C_df)

In [ ]:
#applying the mean
filtered_C_df.mean(axis=0, skipna=True, numeric_only=False)

In [ ]:
#applying standard deviation
filtered_C_df.std(axis=0, skipna=True, ddof=1, numeric_only=False)

In [ ]:
#applying group_by 
grouped_df = df.groupby(['Bromley']).mean()
print(grouped_df)

# Challenge No 2

In [ ]:
# Here is the code for challenge 2, Lab 2

In [ ]:
import requests
import pandas as pd
import geopandas as gpd

# Let's describe the url, it is usually easier to do it like this, so in the future, you can easily update the URL
url_bikes = "https://api.glasgow.gov.uk/mobility/v1/get_rentals?startDate=2022-05-01&endDate=2023-05-01"
# Making the query to the web server, using the Get method from the requests library 
response = requests.get(url_bikes)
response

In [ ]:
#Now we get the response from the web server, we need to translate that into a format we can manipulate, like JSON.
data = response.json()
data
# careful here you will get a huge outcome; explore what you get, and then you can clear this cell outcome

In [ ]:
# Usually, there are two labels into the web server response the metadata, and the data; we will use the data label
# to get all attributes included. 
rental_data = data['data']
rental_data
# See the structure of the data, you can see
# 'attribute':'value' structure
# each {} define one row or one element
# Again, here you will get a huge outcome; just explore what you get, and then you can clear this cell outcome

In [ ]:
rental_pd = pd.DataFrame(rental_data)
rental_pd.head()

In [ ]:
rental_pd.shape

In [ ]:
rental_pd.columns

In [ ]:
# Check for NaN in the coordinates column
nan_in_column_Lat = rental_pd['startPlaceLat'].isna().any()
nan_in_column_Long = rental_pd['startPlaceLong'].isna().any()

print(nan_in_column_Lat,nan_in_column_Lat)

# Alternatively, you can use the following to count NaN values
nan_in_column_Lat = rental_pd['startPlaceLat'].isna().sum()
nan_in_column_Long = rental_pd['startPlaceLong'].isna().sum()
print(nan_in_column_Lat,nan_in_column_Lat)

In [ ]:
clean_rental_pd = rental_pd.dropna(subset=['startPlaceLat', 'startPlaceLong', 'endPlaceLat','endPlaceLong'])
clean_rental_pd.info()

In [ ]:
gdf_bikes_start = gpd.GeoDataFrame(clean_rental_pd, geometry=gpd.points_from_xy(clean_rental_pd['startPlaceLong'], clean_rental_pd['startPlaceLat']))
gdf_bikes_end = gpd.GeoDataFrame(clean_rental_pd, geometry=gpd.points_from_xy(clean_rental_pd['endPlaceLong'], clean_rental_pd['endPlaceLat']))

# Print the GeoDataFrame
gdf_bikes_start.info()
# Do we need all those columns? And you see, there is also a lot of pre-processing to do with all the object Dtype

In [ ]:
#exploring the geodataframe's data types and attributes
gdf_bikes_end.info()

In [ ]:
#exploring the geodataframe
gdf_bikes_end.explore()

In [ ]:
#dropping null values within the subsets start place latitude and longitude
gdf_bikes_end = gdf_bikes_end.dropna(subset=['startPlaceLat', 'startPlaceLong'])

In [ ]:
#setting the coordinate reference system for the geodataframe
gdf_bikes_end = gdf_bikes_end.set_crs("EPSG:4326")

In [ ]:
#keeping columns and cleaning up the dataset
keep_cols = [
    "startDate",
    "startPlaceId",
    "startPlaceName",
    "durationSeconds",
    "isInvalid",
    "price",
    "isEbike",
    "startPlaceLat",
    "startPlaceLong",
    "geometry",
]
gdf_bikes_end = gdf_bikes_end[keep_cols]
gdf_bikes_end.head()

In [ ]:
#now checking how attributes have changed
gdf_bikes_end.info()

In [ ]:
#keeping the data types within the geodataframe consistent
gdf_bikes_end.startPlaceId = gdf_bikes_end.startPlaceId.astype(int)
gdf_bikes_end.startPlaceName = gdf_bikes_end.startPlaceName.astype(str)
gdf_bikes_end['startDate'] = pd.to_datetime(gdf_bikes_end['startDate'], format='%Y-%m-%dT%H:%M:%SZ')

In [ ]:
#checking that the data type consolidation was successful
gdf_bikes_end.dtypes

In [ ]:
#exploring the first few rows of the dataset
gdf_bikes_end.head()

In [ ]:
#importing leafmap in order to generate a map of the geodataframe
import leafmap

m = leafmap.Map(center=(55.860166, -4.257505),
                zoom=12,
                draw_control=False,
                measure_control=False,
                fullscreen_control=False,
                attribution_control=True,
                   
               )

m.add_basemap("CartoDB.Positron")
m.add_data(
    gdf_bikes_end,
    column='startPlaceName',
    legend_title='Clusters',
    cmap='Set1',
    k=4,
)

#Ploting the map
m

In [ ]:
#for part 2 of the challenge, I will describe the url of our data source and use requests to make the query to the web server
import requests
import pandas as pd
import geopandas as gpd

url_sensor = "https://api.glasgow.gov.uk/traffic/v1/movement/sites?null=3_weeks_ago HTTP/1.1"
response = requests.get(url_sensor)
response

In [ ]:
sensor_data = response.json()
sensor_data

In [ ]:
print(sensor_data[0])

In [ ]:
#reference for using Point: Readthedocs.io. (2024). shapely.Point — Shapely 2.0.6 documentation. [online] Available at: https://shapely.readthedocs.io/en/2.0.6/reference/shapely.Point.html.
from shapely.geometry import Point

In [ ]:
#making siteId an integer
for sensor in sensor_data:
    sensor["siteId"] = int(sensor["siteId"])

In [ ]:
#changing siteId
df_zones["siteId"] = pd.to_numeric(df_zones["siteId"], errors="coerce")  

In [ ]:
#focusing on siteIds - extracting locations of sensors for the dataframe
sensor_list = sensor_data["siteId"]
df_sensors = pd.DataFrame(sensor_list)

In [ ]:
#converting the json to a dataframe
gdf_sensors = gpd.GeoDataFrame(
    df_sensors, geometry=gpd.points_from_xy(df_sensors['lon'], df_sensors['lat'])
)

In [ ]:
#setting our CRS to the Geodetic coordinate system
gdf_sensors.set_crs(epsg=4326, inplace=True)

In [ ]:
#exploring the first few rows of data
print(gdf_sensors.head())

In [ ]:
#repeating process for working zones from API URL for Glasgow
url_zones = "https://api.glasgow.gov.uk/traffic/v1/working_zones"

#fetching the data
response_zones = requests.get(url_zones)
response_zones

In [ ]:
zones_data = response.json()
zones_data

In [ ]:
#reference for using Point: Readthedocs.io. (2024). shapely.Point — Shapely 2.0.6 documentation. [online] Available at: https://shapely.readthedocs.io/en/2.0.6/reference/shapely.Point.html.
from shapely.geometry import Point

In [ ]:
#making siteId an integer
for zones in zones_data:
    zones["siteId"] = int(zones["siteId"])

In [ ]:
#converting into dataframe
df_zones = pd.DataFrame(zones_data["siteId"])  
df_zones["geometry"] = df_zones["polygon"].apply(lambda x: Polygon(x)) 

#setting crs and geometry
gdf_zones = gpd.GeoDataFrame(df_zones, geometry="geometry")
gdf_zones.set_crs(epsg=4326, inplace=True)

#exploring the first few rows of data
print(gdf_zones.head())

In [ ]:
#performing spatial joins using sjoin
gdf_sensors_zones = gpd.sjoin(gdf_sensors, gdf_zones, how="inner", predicate="within")

#exploring the first few rows
print(gdf_sensors_zones.head())

In [ ]:
#counting the sensors in every working zone
sensor_counts = gdf_sensors_zones.groupby("zone_id").size().reset_index(name="sensor_count")

#merging the counts into the working zone geodataframe
gdf_zones = gdf_zones.merge(sensor_counts, on="zone_id", how="left")

#getting rid of non values
gdf_zones["sensor_count"] = gdf_zones["sensor_count"].dropna()

print(gdf_zones.head())

In [ ]:
#plotting the chloropleth map using leafmap
import leafmap

m = leafmap.Map(
    center=(56.329031,-3.798943),
    zoom=7
)

m.add_basemap("CartoDB.Positron")

m.add_data(
    gdf_zones,
    column="sensor_count",
    legend_title="Sensor Count",
    cmap="OrRd",  # Red color gradient (adjust if needed)
    k=5,  # Number of color bins
)

m